In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pylab as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from plotly import tools
import scipy.stats as stats


In [2]:
plotly.tools.set_credentials_file(username='alelopes', api_key='ZETQxILVg72WbsKtxKyy')

### Reading the data 

In [3]:
!ls

bank-additional      bank.csv	    bank-names.txt  __MACOSX
bank-additional.zip  bank-full.csv  bank.zip	    Semantix.ipynb


In [4]:
bank = pd.read_csv("bank-full.csv",sep=";",dtype='category')

In [5]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [122]:
bank['age']=bank['age'].astype("int32")
bank['duration']=bank['duration'].astype("int32")
bank['campaign']=bank['campaign'].astype("int32")
bank['pdays']=bank['pdays'].astype("int32")
bank['previous']=bank['previous'].astype("int32")
bank['balance']=bank['balance'].astype("int32")

## First Question

In [7]:
jobs_and_loans = bank[['job','loan','housing']].copy()
jobs_and_loans['loan']=jobs_and_loans['loan'].eq('yes').mul(1)
jobs_and_loans['housing']=jobs_and_loans['housing'].eq('yes').mul(1)
jobs_and_loans['counts'] = 1
jobs_and_loans['loans'] = jobs_and_loans['loan'] | jobs_and_loans['housing'] 
agg=jobs_and_loans.groupby('job').sum()

In [8]:
data = [go.Bar(x=agg.index.get_level_values(0),
            y=agg.loans,name='People with Any kind of Loan'),
       go.Bar(x=agg.index.get_level_values(0),
            y=agg.counts,name='All Contacted People'),
       ]
layout = go.Layout(
    title='People Contacted with Loans and All Contacted by Jobs',
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

PlotlyRequestError: Account limit reached: Your account is limited to creating 25 charts. To continue, you can override or delete existing charts or you can upgrade your account at: https://plot.ly/products/cloud

In [10]:
data = [go.Bar(x=agg.index.get_level_values(0),
            y=agg.loans/agg.counts,name='Loans/All')
       ]
layout = go.Layout(
    title='Ratio of people with loan X all contacted people',
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [11]:
data = [go.Bar(x=agg.index.get_level_values(0),
            y=agg.loan,name='Loan'),
        go.Bar(x=agg.index.get_level_values(0),
            y=agg.housing,name='Housing Loan')
       ]
layout = go.Layout(
    title='Types of Loans by Job',
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

## Second Question

In [8]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [9]:
previous_contacts = bank[['previous','campaign','y']].copy()

In [10]:
previous_contacts.groupby(['y']).describe()

campaign                                               previous            \
       count      mean       std  min  25%  50%  75%   max    count      mean   
y                                                                               
no   39922.0  2.846350  3.212767  1.0  1.0  2.0  3.0  63.0  39922.0  0.502154   
yes   5289.0  2.141047  1.921826  1.0  1.0  2.0  3.0  32.0   5289.0  1.170354   

                                          
          std  min  25%  50%  75%    max  
y                                         
no   2.256771  0.0  0.0  0.0  0.0  275.0  
yes  2.553272  0.0  0.0  0.0  1.0   58.0

In [11]:
previous_contacts['previous']=previous_contacts.previous>0

In [12]:
agg_prev_y =previous_contacts.groupby(['y','previous'],as_index=False).count()

In [13]:
agg_prev_y

,y,previous,campaign
0,no,False,33570
1,no,True,6352
2,yes,False,3384
3,yes,True,1905


In [14]:
agg_prev_y[agg_prev_y['y']=='no']

,y,previous,campaign
0,no,False,33570
1,no,True,6352


In [15]:
agg_prev_y[agg_prev_y['y']=='yes']

,y,previous,campaign
2,yes,False,3384
3,yes,True,1905


In [16]:
ratios=[previous_contacts.groupby(['y','previous']).count().campaign[1]/previous_contacts.groupby(['y','previous']).count().campaign[0:2].sum(),previous_contacts.groupby(['y','previous']).count().campaign[3]/previous_contacts.groupby(['y','previous']).count().campaign[2:4].sum()]

In [141]:
data = [go.Bar(x=['Never Contacted','Previously Contacted'],
            y=agg_prev_y[agg_prev_y['y']=='no'].campaign,name='Subscribed'),
        go.Bar(x=['Never Contacted','Previously Contacted'],
            y=agg_prev_y[agg_prev_y['y']=='yes'].campaign,name='Not Subscribed')
        
               ]
layout = go.Layout(
    title='Subscribed/Not Subscribed X Never Contacted/Contacted in Previous Campaigns',
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

### Third

In [9]:
campaign_y = bank[['campaign','y']].copy()

In [10]:
campaign_y.groupby('y').quantile([0.6,0.7,0.8,0.9,0.95,0.98])

campaign
y                 
no  0.60       2.0
    0.70       3.0
    0.80       4.0
    0.90       6.0
    0.95       8.0
    0.98      13.0
yes 0.60       2.0
    0.70       2.0
    0.80       3.0
    0.90       4.0
    0.95       5.0
    0.98       8.0

In [22]:
excluding_outliers=campaign_y[campaign_y.campaign<13]

In [28]:
excluding_outliers.campaign=excluding_outliers.campaign.astype('category')

/home/alexandre/anaconda3/envs/dl-1.7/lib/python3.5/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [12]:
excluding_outliers.campaign=pd.cut(excluding_outliers.campaign,6,labels=["1 to 2", "3 to 4", "5 to 6", "7 to 8", "9 to 10", "11 to 12"])

/home/alexandre/anaconda3/envs/dl-1.7/lib/python3.5/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [29]:
len(excluding_outliers.campaign.cat.categories.tolist())

12

In [30]:
positive_campaign=excluding_outliers[excluding_outliers['y']=='yes'].groupby('campaign').count()

In [31]:
negative_campaign=excluding_outliers[excluding_outliers['y']=='no'].groupby('campaign').count()

In [32]:
positive_campaign.index.get_level_values(0)

CategoricalIndex([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], categories=[1, 2, 3, 4, 5, 6, 7, 8, ...], ordered=False, name='campaign', dtype='category')

In [38]:
trace1 = go.Scatter(x=positive_campaign.index.get_level_values(0),
            y=positive_campaign.y,name='subscribed',mode = 'lines+markers')


trace2 = go.Scatter(x=negative_campaign.index.get_level_values(0),
            y=negative_campaign.y,name='not subscribed',mode = 'lines+markers')

trace3=go.Scatter(x=negative_campaign.index.get_level_values(0),
            y=positive_campaign.y/(negative_campaign.y+positive_campaign.y),name='ratio',mode = 'lines+markers',yaxis='y2',
                      line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4
        )
                  )
        
       
fig = tools.make_subplots(rows=1, cols=2,subplot_titles=('Subscribed/Not by number of contact', 'Ratio of y True by y for Contacts'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 1, 2)

fig['layout'].update(height=600, width=800, title='Subscribed and Not Subscribed Counts and Ratio ')
py.iplot(fig, filename='subplot')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



## Forfh

In [39]:
poutcome_y = bank[['poutcome','y']].copy()

In [44]:
poutcome_y.groupby('poutcome').describe()

y                   
          count unique  top   freq
poutcome                          
failure    4901      2   no   4283
other      1840      2   no   1533
success    1511      2  yes    978
unknown   36959      2   no  33573

In [58]:
y_no_previous = poutcome_y[poutcome_y.y=='no'].poutcome

In [63]:
y_no_previous.value_counts()

unknown    33573
failure     4283
other       1533
success      533
Name: poutcome, dtype: int64

In [59]:
y_yes_previous = poutcome_y[poutcome_y.y=='yes'].poutcome

In [57]:
y_no_previous.cat.categories.tolist()#index.get_level_values(0)

['failure', 'other', 'success', 'unknown']

In [ ]:
go.Pie(data

In [ ]:
go.Pie(domain=)

In [74]:
fig = {
'data': [
        
        {
           'labels':y_no_previous.cat.categories.tolist(),
            'values':y_no_previous.value_counts(),
            "type": 'pie',
            'name': 'Irises'
        },
        {
           labels=y_no_previous.cat.categories.tolist(),
            values=y_yes_previous.value_counts(),
            'type': 'pie',
            'name': 'Irises2'
        }
    ],
    'layout': {'title': 'Van Gogh: 5 Most Prominent Colors Shown Proportionally',
               'showlegend': False}
}
py.iplot(fig, filename='stacked-bar')


SyntaxError: invalid syntax (<ipython-input-74-7b9cd11b0a0f>, line 11)

In [116]:
trace1 = go.Pie(
    labels=y_no_previous.cat.categories.tolist(),
    values=y_no_previous.value_counts(),
    domain={"x": [0, .48]},
#    text=["lala"],
#    textinfo="text"
    name='y=0'
    
)

trace2 = go.Pie(
    labels=y_no_previous.cat.categories.tolist(),
    values=y_yes_previous.value_counts(),
    domain={"x": [.52, 1]},
    name='y=1'
)

data = [trace1,trace2]
#layout = go.Layout(
#    barmode='stack'
#)
layout = go.Layout(
    annotations=[
        dict(
            x=0.08,
            y=1.05,
            showarrow=False,
            text='Not Subscribed X Previous Campaign Result',
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.92,
            y=1.05,
            showarrow=False,
            text='Subscribed X Previous Campaign Result',
            xref='paper',
            yref='paper'
        )

    ],
    autosize=True,
    margin=dict(
        b=100
    ),
    title='Subscribed/Not Subscribed by Previous Campaign Result.',
    height=600,
    width=950
)


fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename='stacked-bar')

## Fifth

In [209]:
for col in bank.columns:
    print(col,bank['balance'].corr(bank[col],method= 'spearman'))




age 0.096380304772


/home/alexandre/anaconda3/envs/dl-1.7/lib/python3.5/site-packages/scipy/stats/stats.py:245: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.



job 0.0296089703645
marital 0.0202814404841
education 0.0753277972862
default -0.167738798338
balance 1.0
housing -0.0682919560213
loan -0.128966094322
contact -0.0342447210848
day -0.0503725884809
month 0.0275751312943
duration 0.0426506809879
campaign -0.0309591222681
pdays 0.0696756838137
previous 0.0795357763866
poutcome -0.0753749761313
y 0.100295315069


In [208]:
bank.columns.tolist()

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y']

In [200]:
n_table = bank[['default','balance']].copy()

In [194]:
n_table.balance=pd.cut(n_table.balance,20,labels=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19'])

In [201]:
n_table

,default,balance
0,no,2143
1,no,29
2,no,2
3,no,1506
4,no,1
5,no,231
6,no,447
7,yes,2
8,no,121
9,no,593


In [206]:
n_table[n_table.default=="yes"].balance.describe()

count      815.000000
mean      -137.624540
std        906.704678
min      -8019.000000
25%       -339.000000
50%         -7.000000
75%         15.000000
max      16486.000000
Name: balance, dtype: float64

In [207]:
n_table[n_table.default=="no"].balance.describe()

count     44396.000000
mean       1389.806424
std        3063.275425
min       -4057.000000
25%          84.000000
50%         468.000000
75%        1462.250000
max      102127.000000
Name: balance, dtype: float64

In [183]:
for col in n_table.columns:
    print(col,n_table['default'].corr(n_table[col],method= 'spearman'))

default 1.0
balance -0.0494805240682


/home/alexandre/anaconda3/envs/dl-1.7/lib/python3.5/site-packages/scipy/stats/stats.py:245: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.



In [167]:
for col in bank.columns:
    print(col,bank['housing'].corr(bank[col],method= 'spearman'))

/home/alexandre/anaconda3/envs/dl-1.7/lib/python3.5/site-packages/scipy/stats/stats.py:245: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.



age -0.15434020548
job -0.131364144171
marital -0.0170466399999
education -0.0943944346027
default -0.00602521835625
balance -0.0682919560213
housing 1.0
loan 0.0413228660413
contact 0.178191435857
day -0.0484994266708
month 0.265319913509
duration 0.0051871062652
campaign -0.0378067792573
pdays 0.0809770393243
previous 0.0620874304328
poutcome -0.074550664124
y -0.139172702498


In [278]:
la=[10,300,100,100,500,20,30]

In [299]:
la=['fe','f','fe','f','fe']

In [279]:
lo=['no','yes','yes','yes','yes','no','no']

In [300]:
lo=['no','yes','yes','yes','yes']

In [301]:
pd.Series(la).corr(pd.Series(lo),method='spearman')

/home/alexandre/anaconda3/envs/dl-1.7/lib/python3.5/site-packages/scipy/stats/stats.py:245: RuntimeWarning:

The input array could not be properly checked for nan values. nan values will be ignored.



-0.40824829046386302